In [ ]:
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Input experimental data

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Part 1')

# Least Squares

In [ ]:
def lstsqb(x, y):
    return (x.multiply(y).mean() - x.mean() * y.mean()) / (x.multiply(x).mean() - x.mean() ** 2)

In [ ]:
def lstsqa(x, y):
    return y.mean() - lstsqb(x, y) * x.mean()

In [ ]:
def lstsqbsem(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt((y.multiply(y).mean() - y.mean() ** 2) / (x.multiply(x).mean() - x.mean() ** 2) - lstsqb(x, y) ** 2)

# Part 1

## General Data

In [ ]:
d0 = 0.73e-1
r0 = 13e-1
l0 = 176.5
h0 = 139.9
g0 = 9.81

In [ ]:
sigma_n_sys = 0.1
sigma_d_sys = 0.1e-2
sigma_r_sys = 0.1e-1
sigma_l_sys = 0.1

In [ ]:
s = (np.pi * d0 ** 2) / (4)
sigma_s = s * np.sqrt(2 * (sigma_d_sys / d0) ** 2)

In [ ]:
n0 = df1.loc[0, 'n1' : 'n6'].mean()

In [ ]:
df1['P, Н'] = df1.loc[1:, 'm, кг'].multiply(g0)
df1['dn, см'] = df1.loc[1:, 'n1' : 'n6'].apply(lambda x: x.mean() - n0, axis = 1)
df1['dl, см'] = df1.loc[1:, 'dn, см'].apply(lambda x: x * r0 / (2 * h0))

In [ ]:
sigma_n0 = np.sqrt(df1.loc[0, 'n1' : 'n6'].sem() ** 2 + sigma_n_sys ** 2)

In [ ]:
df1['sigma_l, см'] = df1['dl, см'].multiply(np.sqrt((sigma_d_sys / d0) ** 2 + (sigma_r_sys / r0) ** 2 + ((df1.loc[:, 'n1' : 'n6'].sem(axis = 1) ** 2 + sigma_n_sys ** 2 + sigma_n0 ** 2) / (df1['dn, см'] ** 2))))

In [ ]:
df1['E, Н / см^2'] = (df1.loc[0: , 'P, Н'] / df1.loc[0: , 'dl, см'] * l0 / (np.pi * d0 ** 2 / 4))

In [ ]:
k = lstsqb(df1.loc[1: , 'dl, см'], df1.loc[1: , 'P, Н'])
a = lstsqa(df1.loc[1: , 'dl, см'], df1.loc[1: , 'P, Н'])
e = k * l0 / s
sigma_k = lstsqbsem(df1.loc[1: , 'dl, см'], df1.loc[1: , 'P, Н'])
sigma_e = e * np.sqrt((sigma_k / k) ** 2 + (sigma_s / s) ** 2 + (sigma_l_sys / l0) ** 2)

# Results

In [ ]:
k

In [ ]:
sigma_k

In [ ]:
e

In [ ]:
sigma_e

# Tables

In [ ]:
html1 = df1.to_html('plots/table1.html')

# Plots

In [ ]:
dl_max = 9 / 8 * max(df1.loc[1:, 'dl, см'])
p_max = 9 / 8 * max(df1.loc[1:, 'P, Н'])

In [ ]:
x = np.linspace(0, dl_max, 1000)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

ax.grid(b = True, linestyle = '--')

plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))

ax.set_ylim(bottom = 0, top = p_max)
ax.set_xlim(left = 0, right = dl_max)

plt.gca().xaxis.set_major_locator(plt.MultipleLocator(5e-3))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(2e0))

plt.xlabel('$\Delta{l}, см$')
plt.ylabel('$P, Н$')

ax.errorbar(df1.loc[1:, 'dl, см'], df1.loc[1:, 'P, Н'], xerr = df1.loc[1:, 'sigma_l, см'], fmt = '.')
ax.plot(x, a + k * x, label = '$f(\Delta{l}) = a + \Delta{l} \cdot k$')

ax.set_title('Рис. 1. Зависимость $P(\Delta{l})$')
ax.legend()

fig.savefig('plots/plot1.pdf')

# Part 2

In [ ]:
l2 = 503

In [ ]:
sigma_y_sys = 1e-2
sigma_a_sys = 1e-1
sigma_b_sys = 1e-1
sigma_l2_sys = 1

In [ ]:
df2 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Part 2 (dimensions)')

In [ ]:
dimensions = {}
materials = ['wood', 'steel', 'brass']
for m in materials:
    dimensions[m] = (dict({'mean':{'a': df2[m + '_a, мм'].mean(), 'b': df2[m + '_b, мм'].mean()},
                          'sigma': {'a': df2[m + '_a, мм'].sem(), 'b': df2[m + '_b, мм'].sem()}}))

In [ ]:
df3 = {}
for m in materials:
    df3[m] = pd.read_excel('experimental_data.xlsx', sheet_name = 'Part 2 (' + m + ')')

In [ ]:
for m in materials:
    df3[m]['P, Н'] = df3[m]['m, кг'] * g0
    df3[m]['y_mean, мм'] = df3[m].loc[:, 'y_1, мм' : 'r_y_2, мм'].apply(lambda x: x.mean(), axis = 1)
    df3[m]['sigma_y, мм'] = np.sqrt(sigma_y_sys ** 2 + df3[m].loc[:, 'y_1, мм' : 'r_y_2, мм'].apply(lambda x: x.sem(), axis = 1) ** 2)

In [ ]:
linefit = {}
for m in materials:
    linefit[m] = dict({'mean': {'a': lstsqa(df3[m]['y_mean, мм'], df3[m]['P, Н']), 'k': lstsqb(df3[m]['y_mean, мм'], df3[m]['P, Н'])}
                      ,'sigma': {'k': lstsqbsem(df3[m]['y_mean, мм'], df3[m]['P, Н'])}})

In [ ]:
e_dict = {}
for m in materials:
    t = linefit[m]['mean']['k'] * l2 ** 3 / (4 * dimensions[m]['mean']['a'] * dimensions[m]['mean']['b'] ** 3)
    e_dict[m] = dict({'mean': t, 'sigma': t * np.sqrt(3 * (sigma_l2_sys / l2) ** 2 + (linefit[m]['sigma']['k'] / linefit[m]['mean']['k']) ** 2 + (dimensions[m]['sigma']['a'] / dimensions[m]['mean']['a']) ** 2 + 3 * (dimensions[m]['sigma']['b'] / dimensions[m]['mean']['b']) ** 2)})

# Results

In [ ]:
e_dict

# Tables

In [ ]:
html2 = df3['wood'].to_html('plots/table2.html')
html3 = df3['steel'].to_html('plots/table3.html')
html4 = df3['brass'].to_html('plots/table4.html')

# Plots

In [ ]:
x = np.linspace(0, 5 / 4 * max(max(df3['wood']['y_mean, мм']), max(df3['steel']['y_mean, мм']), max(df3['brass']['y_mean, мм'])))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows = 2, ncols = 2, figsize=(11.7, 8.3))
    
fig.delaxes(ax4)

ax1.grid(b = True, linestyle = '--')
ax2.grid(b = True, linestyle = '--')
ax3.grid(b = True, linestyle = '--')

ax1.errorbar(df3['wood']['y_mean, мм'], df3['wood']['P, Н'], xerr = df3['wood']['sigma_y, мм'], fmt = '.')
ax2.errorbar(df3['steel']['y_mean, мм'], df3['steel']['P, Н'], xerr = df3['steel']['sigma_y, мм'], fmt = '.')
ax3.errorbar(df3['brass']['y_mean, мм'], df3['brass']['P, Н'], xerr = df3['brass']['sigma_y, мм'], fmt = '.')

ax1.plot(x, linefit['wood']['mean']['a'] + linefit['wood']['mean']['k'] * x, label = '$f(y) = a + y \cdot k$')
ax2.plot(x, linefit['steel']['mean']['a'] + linefit['steel']['mean']['k'] * x, label = '$f(y) = a + y \cdot k$')
ax3.plot(x, linefit['brass']['mean']['a'] + linefit['brass']['mean']['k'] * x, label = '$f(y) = a + y \cdot k$')

ax1.set_ylim(bottom = 0, top = max(5 / 4 * df3['wood']['P, Н']))
ax1.set_xlim(left = 0, right = max(5 / 4 * df3['wood']['y_mean, мм']))

ax2.set_ylim(bottom = 0, top = max(5 / 4 * df3['steel']['P, Н']))
ax2.set_xlim(left = 0, right = max(5 / 4 * df3['steel']['y_mean, мм']))

ax3.set_ylim(bottom = 0, top = max(5 / 4 * df3['brass']['P, Н']))
ax3.set_xlim(left = 0, right = max(5 / 4 * df3['brass']['y_mean, мм']))

ax1.xaxis.set_major_locator(plt.MultipleLocator(1))
ax2.xaxis.set_major_locator(plt.MultipleLocator(1))
ax3.xaxis.set_major_locator(plt.MultipleLocator(1))

ax1.set_title('Рис. 2. Зависимость $P(y)$ (дерево)')
ax2.set_title('Рис. 3. Зависимость $P(y)$ (сталь)')
ax3.set_title('Рис. 4. Зависимость $P(y)$ (латунь)')

ax1.legend()
ax2.legend()
ax3.legend()

fig.savefig('plots/plot2.pdf')